# Pipeline Stage 3
The messages in what I called the "mq_sgcn_items" message queue contain records from ScienceBase that should have a new file to process. Those files contain names of species that the states/territories consider to be of greatest conservation need. There is some messiness in the data as there are cases where there may not be known scientific names or other information has been added to the spreadsheets that can get in the way of nice, clean processing to align the names with taxonomic authorities and put together other details. The initial processing step that slurps up files is encapsulated in the process_sgcn_source_item function from the pysgcn package. It picks up the messages, reads the specified file via URL into memory, and does some basic processing to harmonize across the slightly variable files as much as possible and infuse a little bit of additional metadata from the root collection item.

The two pieces of additional information the function infuses come from files attached to the root collection. These include the following:

* An indication of whether the name was included in the 2005 SWAP-based list coming from a master list stored from original processing. This helps us maintain consistency in total counts of species.
* A number of names had to be tracked down with a bit of research to assign ITIS TSN identifiers. These serve as overrides to the name matching process.

Each dataset can be processed at this stage and stand on its own somewhere. The point here is to get the original data file from ScienceBase, making sure we can read it into memory as a dataframe, infusing the extra information from the reference files stored at the collection, and then stashing the data somewhere online for further processing in later steps. I put all of that logic together into the cache_item_data function that calls the more fundamental processing function, process_sgcn_source_item. The caching function will check the cache when called to make sure the cache is intact and then delete the message if that's the case. Or it will go ahead and fire the processing function to retrieve and process the file and then write it to cache. For this exercise, I cache the data to a folder within the space designated by the DATA_CACHE environment variable as a feather binary file. These are lightweight, fast, and easy to work with across programs. For the online instantiation of this workflow, we will probably want to go ahead and write them to a relational database as we can later assemble them from there into final usable data.

I run the process as a while loop on the messages in temporary storage, but this will run as messages that get flushed from the queue as lambdas operate on them.

Note: I have experienced some variation in the speed of this step, which seem to track back to ScienceBase variability.

In [1]:
import pysgcn
sgcn = pysgcn.sgcn.Sgcn()

In [2]:
processable_item = sgcn.get_message("mq_sgcn_items")
processable_item

{'id': '161910363f73ac67bed0c5ba86b1a09f7e761c52',
 'date_inserted': '2019-12-20T14:03:55.646715',
 'body': {'sciencebase_item_id': 'https://www.sciencebase.gov/catalog/item/5787cc52e4b0d27deb3754de',
  'state': 'Idaho',
  'year': '2005',
  'source_file_url': 'https://www.sciencebase.gov/catalog/file/get/5787cc52e4b0d27deb3754de?f=__disk__e9%2F69%2F0e%2Fe9690e1a26d17f001245f4199396cccaea7fb7fe',
  'source_file_date': '2017-06-22T21:28:05.000Z'}}

In [3]:
%%time
while processable_item is not None:
    sgcn.cache_item_data(processable_item["body"])
    sgcn.delete_message("mq_sgcn_items", processable_item["id"])
    processable_item = sgcn.get_message("mq_sgcn_items")

URLError: <urlopen error [Errno 60] Operation timed out>